# Imports

In [1]:
from matplotlib import pyplot as plt
from transformers import pipeline
import pandas as pd

# Для более качественных графиков
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 150

# Huggingface

[Документация](https://huggingface.co/docs/transformers/quicktour)

In [ ]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


In [35]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

Можем довольно просто получить эмбеддинг текста

In [6]:
embed_model = pipeline("feature-extraction")

embed_model("Oh hi mark")

No model was supplied, defaulted to distilbert/distilbert-base-cased and revision 6ea8117 (https://huggingface.co/distilbert/distilbert-base-cased).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[[[0.45091745257377625,
   0.14973433315753937,
   0.16388116776943207,
   -0.24116627871990204,
   -0.19994936883449554,
   -0.11540158092975616,
   0.26028645038604736,
   -0.016589652746915817,
   0.09965295344591141,
   -1.1254569292068481,
   -0.22353047132492065,
   0.24460867047309875,
   -0.09785911440849304,
   0.007572060450911522,
   -0.5261136889457703,
   0.2617356479167938,
   -0.11085841804742813,
   0.10818532109260559,
   -0.27364587783813477,
   -0.054224807769060135,
   0.20001263916492462,
   -0.2963610887527466,
   0.4895520806312561,
   -0.3139702379703522,
   0.26250478625297546,
   0.07292275875806808,
   0.22152379155158997,
   0.20509925484657288,
   -0.1394108235836029,
   0.3074868321418762,
   0.20792590081691742,
   0.1675594300031662,
   -0.04460762068629265,
   -0.040615543723106384,
   -0.2277088463306427,
   0.010573580861091614,
   -0.01782449334859848,
   -0.12859955430030823,
   -0.01604573428630829,
   -0.24653367698192596,
   -0.4732385277748108,


Задание: 
- выделить трейн и тест выборки на датасете anime_reviews.csv, доля теста 0.2, random_state 42
- сделать предсказание тональности на тестовой выборке при помощи [модели](https://huggingface.co/tabularisai/multilingual-sentiment-analysis), посчитать f1

In [3]:
df = pd.read_csv("data/anime_reviews.csv")
df

,Anime,Text,Sentiment
0,Граница пустоты 5: Спираль парадокса / Kara no...,"Вселенная ""Nasuverse"" одна из моих любимых в а...",1
1,"Красный, белый, чёрный, жёлтый: Ледяная импери...",Что всегда делало RWBY таким особенным для мен...,0
2,Любовь похожа на прошедший дождь / Koi wa Amea...,Есть у этого аниме интересная завязка - влюбле...,0
3,Шальные девчонки / Girls of the Wild's,"260 глав пролетели в мгновение ока, в душе оса...",1
4,Ангел кровопролития / Satsuriku no Tenshi,"Сначала я подумала что реально ужасы, будет чт...",0
...,...,...,...
475,Ложные выводы / Kyokou Suiri,Неприятно уникальный детектив(с жуткой поправк...,1
476,Твоё имя / Kimi no Na wa.,"Напишу сначала о хорошем: музыка, визуал, техн...",0
477,Супербои в повседневности / Inou-Battle wa Nic...,"Че то последние отзывы идут не очень, надо бы ...",1
478,Токийский гуль: Перерождение 2 / Tokyo Ghoul:r...,"Я рад, что история Канеки наконец то закончена...",0


In [55]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

X = df["Text"]
y = df["Sentiment"]

model = pipeline("sentiment-analysis", model="tabularisai/multilingual-sentiment-analysis")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

output_map = {
    "LABEL_0": 0,
    "LABEL_1": 0,
    "LABEL_2": 1,
    "LABEL_3": 1,
    "LABEL_4": 1,
}

preds = []
for t in tqdm(X_test):
    output = model(t[:500])
    preds.append(output_map[output[0]["label"]])

Device set to use mps:0
100%|██████████| 96/96 [00:49<00:00,  1.96it/s]


In [56]:
f1_score(y_true=y_test, y_pred=preds)

0.7833333333333333

Аналогично эмбеддингам word2vec, эмбеддинги берта можно использовать как признаки для обучения